# Aufgabe 25

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from numpy.polynomial.polynomial import polyval
import uncertainties.unumpy as unp
from uncertainties.unumpy import nominal_values as noms
from uncertainties.unumpy import std_devs as stds
import pandas as pd

__a)__ Bestimme die Parameter mit der Methoder der kleinsten Quadrate:

In [ ]:
#read data
x, y = np.genfromtxt('aufg_a.csv', delimiter = ',', unpack = True)

#design matrix
A = np.array([x**i for i in range(7)]).T

#parameters with least square
best_a = np.linalg.inv(A.T @ A) @ A.T @ y

for i in range(7):
    print(f'a_{i} ~ {best_a[i]:.4f}')

Stelle das Ergebnis graphisch dar:

In [ ]:
xplot = np.linspace(x[0], x[-1], 100)
plt.plot(xplot, polyval(xplot, best_a), label = 'Fit')
plt.plot(x, y, '.', label = 'Daten')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

__b)__ Erstelle zunächst die Matrix $C$, mit der die numerische zweite Ableitung bestimmt wird:

In [ ]:
C = np.zeros((np.shape(A)[0], np.shape(A)[0]))
np.fill_diagonal(C, -2)
np.fill_diagonal(C[1:], 1)
np.fill_diagonal(C[:, 1:], 1)
C[0, 0] = -1
C[-1, -1] = -1
C

Gibt es dafür eine fertige Methode?

Stelle die Ergebnisse der Regularisierung  für verschiedene $\lambda$ dar:

In [ ]:
plt.plot(x, y, '.', label = 'Daten')

for lam in [0.1, 0.3, 0.7, 3, 10]:
    gamma = np.sqrt(lam) * C @ A
    best_a_reg = np.linalg.inv(A.T @ A + gamma.T @ gamma) @ A.T @ y  
    plt.plot(xplot, polyval(xplot, best_a_reg), 
             label = f'$\lambda = {lam}$')
    
plt.legend()    
plt.show()

__c)__ 

In [ ]:
#read data
data = pd.read_csv('aufg_c.csv')
x = data['x']

#calculate mean and error for y
y = unp.uarray(data.drop(columns = 'x').T.mean(), 
               data.drop(columns = 'x').T.std())

#weight matrix
W = np.zeros((np.shape(A)[0], np.shape(A)[0]))
np.fill_diagonal(W, 1 / stds(y)**2)

In [ ]:
#calculate parameters
best_a_weight = np.linalg.inv(A.T @ W @ A) @ A.T @ W @ noms(y)

Stelle Ergebnisse in einem Plot dar:

In [ ]:
plt.plot(xplot, polyval(xplot, best_a_weight), label = 'Gewichteter Fit')
plt.errorbar(x = x, y = noms(y), 
             yerr = stds(y), 
             label = 'Daten', marker = '.', linestyle = '')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()